# Projeto 1 - Ciência dos Dados

Nome: Evilyn Araujo Vitoraci

Nome: Max

Nome: Taina Nascimento Barreto

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

In [4]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Max


Carregando a base de dados com as mensagens dos seus arquivos:

In [5]:
train = pd.read_csv('dados_treino_TRIO_Max.csv')
train.head(5)

,Comment,Topic
0,My best guess is the table was lightly vibrati...,Physics
1,"I didn't check the molar mass of Al2O3, but th...",Chemistry
2,You could also put it in terms of light wave l...,Biology
3,But what kind of cells are replaced everyday? ...,Biology
4,"It's biological agent A, use bleach to kill. FTFY",Biology


In [6]:
test = pd.read_csv('dados_teste_TRIO_Max.csv')
test.head(5)

,Comment,Topic
0,"I’d go for it, but I’d also apply for the PhD ...",Chemistry
1,"Spiders aren't just harmless, they are benefic...",Biology
2,Yup! It’s r/whatsthisbug,Biology
3,I probably did close to 100 during my first jo...,Chemistry
4,It’s a kind of magic,Physics


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.



___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [27]:
def limpa_txt(text):
    #define o que conta como caracteres necessários para formar palavras (não necessariamente apenas as letras, no inglês o ' por exemplo também é um caractere essencial)
    alfabeto = ['q','w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','i','j','k','l','z','x','c','v','b','n','m', ' ', '’' ,"’s", "’m", "’t", "’n", "'"]
    txt = text.lower() #coloca todo o texto em letras minúsculas para facilitar limpagem
    for word in txt:     
        for l in word:
            if l not in alfabeto:
            #vai de letra em letra no texto e se a letra não estiver na lista "alfabeto" ela é substituida por um espaço em branco
                txt = txt.replace(l, '')
    return txt    
def removeN(text):
    #substitui prefixos e sufixos comuns mas desnescessários (quebras de linha, começo de urls, significadores de subreddits) por espaços em branco
    #ao mesmo tempo também aplica uma remoção de stop-words, nominalmente a remoção de siglas cujo significado não pesam significantemente em nenhuma das matérias analisadas
    #onomatopeias e pronomes e preposições que não pesam na classificação de uma frase (expressões comuns o suficiente ao ponto de atrapalhar a classificação mais do que ajudar)
    #e também foi aplicada lematização, que é o agrupamento de palavras com o mesmo significado para serem contadas como uma só.
    #https://web.archive.org/web/20240222071715/https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/#expand
    txt = text.lower() 
    txt = txt.replace('https://', ' ')
    txt = txt.replace('\\n', ' ')
    txt = txt.replace('\n', ' ')
    txt = txt.replace('\n\ ', ' ')
    txt = txt.replace('n\ ', ' ')
    txt = txt.replace('r/', ' ')
    txt = txt.replace('.jpeg', ' ')
    txt = txt.replace('http', ' ')
    txt = txt.replace('.com', ' ')
    txt = txt.replace('www.', ' ')
    txt = txt.replace("i m ", " i’m ")
    txt = txt.replace("iam ", " i’m ")
    txt = txt.replace("youre", "  ")
    txt = txt.replace("dont", " don't ")
    txt = txt.replace("don t ", " don't ")
    txt = txt.replace("deleted", "removed")
    txt = txt.replace(" bs ", " ")
    txt = txt.replace("lol", " ")
    txt = txt.replace("lmao", " ")
    txt = txt.replace("lmfao", " ")
    txt = txt.replace(" idk ", " ")
    txt = txt.replace("idfk", " ")
    txt = txt.replace("idgf", " ")
    txt = txt.replace(" ake", " ")
    txt = txt.replace("aka", " ")
    txt = txt.replace("neature", "nature")
    txt = txt.replace(" ugh", "  ")
    txt = txt.replace("reddit", " social media ")
    txt = txt.replace(" thru ", " through ")
    txt = txt.replace(" thro ", " through ")
    txt = txt.replace(" tru ", " true ")
    txt = txt.replace("imo", " ")
    txt = txt.replace("opnion", " opinion ")
    txt = txt.replace(" hw ", " homework ")
    txt = txt.replace(" sophomore ", " high school ")
    txt = txt.replace("college", " university ")
    txt = txt.replace("cursed", " magic ")
    txt = txt.replace(" the ", "  ")
    txt = txt.replace(" was ", "  ")
    txt = txt.replace("yup", " yes ")
    txt = txt.replace("yeah", " yes ")
    txt = txt.replace("yep", " yes ")
    txt = txt.replace("truenk", " trunk ")
    txt = txt.replace("universities", " university ")
    txt = txt.replace("facebook", " social media ")
    txt = txt.replace(" sub ", " ")
    txt = txt.replace(" v ", " very ")
    txt = txt.replace("thread ", "page")
    txt = txt.replace(" site ", "page")
    txt = txt.replace(" us ", " ")
    txt = txt.replace(" is ", " ")
    txt = txt.replace("mods", " mod ")
    txt = txt.replace("moderator", " mod ")
    txt = txt.replace("moderators", " mod ")
    txt = txt.replace("ive ", " ")
    txt = txt.replace("i’ve ", " ")
    txt = txt.replace("i’m", " ")
    txt = txt.replace(" ama ", " ")
    txt = txt.replace(" amas ", " ")
    txt = txt.replace(" u r ", "  ")
    txt = txt.replace(" ur ", "  ")
    txt = txt.replace(" hi ", " hello ")
    txt = txt.replace("yikes ", " ")
    txt = txt.replace("omg ", " ")
    txt = txt.replace(" lab ", "laboratory")
    txt = txt.replace("carbs ", " carbohydrates ")
    txt = txt.replace("iirc ", " ")
    txt = txt.replace("thanks", " thank you ")
    txt = txt.replace("master's degree", " masters ")
    txt = txt.replace(" ba ", " bachelor's ")
    txt = txt.replace("imho ", " ")
    txt = txt.replace(" a ", " ")
    txt = txt.replace(" neither ", " ")
    txt = txt.replace(" nor ", " ")
    txt = txt.replace(" else ", " ")
    txt = txt.replace(" so on ", " ")
    txt = txt.replace(" you ", " ")
    txt = txt.replace(" you're ", " ")
    txt = txt.replace(" as ", " ")
    txt = txt.replace("stupid", " dumb ")
    txt = txt.replace(" ez ", " easy ")
    txt = txt.replace("spider", "spiders")  
    txt = txt.replace(" of ", " ")
    return txt

In [28]:
base_bio = train.Topic == 'Biology' #cria filtro para separar apenas as partes da database que compartilham o tópico "Biology"
base_fis = train.Topic == 'Physics' #cria filtro para separar apenas as partes da database que compartilham o tópico "Physics"
base_chem = train.Topic == 'Chemistry' #cria filtro para separar apenas as partes da database que compartilham o tópico "Chemistry"


#aplica primeiro a função removeN para tirar partes comuns que misturam simbolos com caracteres para evitar problemas na hora de fazer a limpagem própria com o limpa_txt


biol = train.loc[base_bio, :].Comment.apply(removeN).apply(limpa_txt).to_list() 
fisi = train.loc[base_fis, :].Comment.apply(removeN).apply(limpa_txt).to_list()
chem = train.loc[base_chem, :].Comment.apply(removeN).apply(limpa_txt).to_list()

#junta as listas com os textos das databases já filtrados numa única string para cada assunto

biol = ' '.join(biol)
fisi = ' '.join(fisi)
chem = ' '.join(chem)

In [29]:
#separa a string para que agora cada item seja uma palavra e não uma frase

bioli = biol.split()
fisic = fisi.split()
chemi = chem.split()

#transforma cada lista numa série para facilitar manipulação dos dados

serie_bio = pd.Series(bioli)
serie_fisic = pd.Series(fisic)
serie_chem = pd.Series(chemi)

#Cria tabelas de frequência absolutas para cada matéria que virá a ser útil em situações onde uma palavra não aparece numa determinada série mas aparece em outras

fisic_tabela = serie_fisic.value_counts()  
bio_tabela = serie_bio.value_counts() 
chem_tabela = serie_chem.value_counts() 

ptbr = biol + ' ' + fisi + ' ' + chem #juntar todas as palavras nas databases de química, física e biologia para definir o espaço amostral de linguagem
all_words = ptbr.split()
port = pd.Series(all_words)
port_t = port.value_counts() # tabela de frequencias absolutas de todas as palavras
port_r = port.value_counts(True) # tabela de frequencias relativas de todas as palavras

In [30]:
#test.Comment.apply(removeN).apply(limpa_txt).tolist()

In [31]:
#Aplicando a limpeza na base de teste:
teste = test.Comment.apply(removeN).apply(limpa_txt)

In [32]:
#Calculando as probablidades de biologia, física e química, respectivamente: 

P_bio = bio_tabela.sum()/ port_t.sum()
P_fis = fisic_tabela.sum()/ port_t.sum() 
P_chem = chem_tabela.sum()/port_t.sum()

print(f'{P_bio}\n{P_fis}\n{P_chem}')

0.39511082732828606
0.28726625923561067
0.31762291343610327


In [33]:
#Importante para suavização de Laplace, total de palavras ÚNICAS:
V = len(port_t) 

#Criando uma lista para guardar os valores:
prob_frases = []

#Varrendo cada frase dentro do arquivo teste:
for frase in teste:
    #Separando as palavras:
    frase_split = (' '.join(frase.split())).split()
    
    #Para o cálculo das probabilidades:
    #P(A|B) = P(B|A) * P(A) / P(B)
    #P(B|A) = produto das probabilidades de cada palavra dado (matéria)
    #Por isso, a probabilidade inicial será igual a probabilidade de P(A)
    
    #Sabendo que a prob. da frase é o produto das prob. de todas as palavras da frase,
    #caso uma palavra não tenha chances de se encaixar em nenhuma das classificações, 
    #a prob da frase seria zerada. Por isso, aplica-se o método da laplace, incluindo no cáculo
    #o total de palavras diferentes em todo o conjunto de dados.
    
#--------------------------------------------------------------------------#
    #Calculando a probabilidade da palavra ser classificada em biologia:
    prob_bio = P_bio
    for palavra in frase_split:
        if palavra not in bio_tabela:
            prob_bio *= 1/(bio_tabela.sum()+V)
        else:    
            prob_bio *= (bio_tabela[palavra]+1)/(bio_tabela.sum()+V)

#--------------------------------------------------------------------------#
    #Calculando a probabilidade da palavra ser classificada em física:
    prob_fis = P_fis
    for palavra in frase_split:
        if palavra not in fisic_tabela:
            prob_fis *= 1/(fisic_tabela.sum()+V)
        else:    
            prob_fis *= (fisic_tabela[palavra]+1)/(fisic_tabela.sum()+V)

#--------------------------------------------------------------------------#
    #Calculando a probabilidade da palavra ser classificada em química:
    prob_chem = P_chem
    for palavra in frase_split:
        if palavra not in chem_tabela:
            prob_chem *= 1/(chem_tabela.sum()+V)
        else:    
            prob_chem *= (chem_tabela[palavra]+1)/(chem_tabela.sum()+V)
#--------------------------------------------------------------------------#   
    #Comparando os valores das probabilidades e classificando cada palavra em seu grupo de matéria:
    materia = ''
    if prob_fis > prob_bio and prob_fis > prob_chem:
        materia = 'Physics'
        prob_frases.append(materia)
    elif prob_bio > prob_chem and prob_bio > prob_fis: 
        materia = 'Biology'
        prob_frases.append(materia)
    elif prob_chem > prob_bio and prob_chem > prob_fis:
        materia = 'Chemistry'
        prob_frases.append(materia)
    #Para que as frases que não foram classificadas em alguma matéria estejam incluídas na contagem e o código rode sem problemas, incluímos elas em uma matéria (química):
    else:
        materia='Chemistry'
        prob_frases.append(materia)
        
#Criando uma coluna para classificação das frases:
test['classificacao'] = prob_frases
#visualização do resultado:
test.head(15)

,Comment,Topic,classificacao
0,"I’d go for it, but I’d also apply for the PhD ...",Chemistry,Chemistry
1,"Spiders aren't just harmless, they are benefic...",Biology,Biology
2,Yup! It’s r/whatsthisbug,Biology,Biology
3,I probably did close to 100 during my first jo...,Chemistry,Chemistry
4,It’s a kind of magic,Physics,Biology
5,Short answer: no. \n\nLess short answer: ignor...,Physics,Physics
6,I’m a man child if that counts,Biology,Biology
7,&#x200B;\n\n||Robert||\n|:-|:-|:-|\n|Emily|Mf-...,Biology,Physics
8,I think this is my favorite thread on Reddit. ...,Biology,Biology
9,r/askphysics,Physics,Physics


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [34]:
#teste e vizualização dos resultados:
tab_performance = pd.crosstab(test['Topic'],test['classificacao'],normalize=True)
acuracia = tab_performance['Biology']['Biology'] + tab_performance['Chemistry']['Chemistry'] + tab_performance['Physics']['Physics']
print(acuracia)
tab_performance

0.7073333333333334


classificacao,Biology,Chemistry,Physics
Topic,,,
Biology,0.319333,0.060000,0.026667
Chemistry,0.070000,0.228667,0.029333
Physics,0.045333,0.061333,0.159333


In [35]:
pd.crosstab(test['Topic'],test['classificacao'],normalize='index')

classificacao,Biology,Chemistry,Physics
Topic,,,
Biology,0.786535,0.147783,0.065681
Chemistry,0.213415,0.697154,0.089431
Physics,0.170426,0.230576,0.598997


___
### Concluindo

### Outra possibilidade de uso para o Naïve : 
O Naïve Bayes também pode ser utilizado para detectar mensagens que ferem os direitos humanos em redes sociais, ao procurar, por exemplo, por palavras ofensivas e que expressam algum tipo de preconceito ou expressão de ódio. 
Referência: "https://blog.somostera.com/data-science/naive-bayes"

Além disso, partindo de uma base de dados adequada, o classificador Naïve Bayes pode ser usado para analisar comentários e separá-los em grupos de prossiveis sentimentos, se a avaliação foi positiva ou negativa, por exemplo.
Referência: https://rockcontent.com/br/blog/naive-bayes/

### Por que novas mensagens classificadas pelo próprio classificador não podem ser usadas como amostra de treinamento?
 Como a acurácia do classificador tem uma margem de erro não negligível, é possível e esperado que ele classifique ocasionalmente textos de maneira errada; se um desses textos erroneamente classificados se juntasse à amostra de treinamento, ele incluiria palavras, termos e frequências que não pertencem à categoria em que ele foi incluso junto às da categoria, o que poluiria o cálculo das probabilidades e diminuiria a sua acurácia mais ainda.



### Reflexão:
Como já citado anteriormente, nosso classificador consegue apresentar resultados cuja probabilidade de estarem certos não chegam a 100%, isso se deve a alguns fatores, como: uma base de dados limitada e filtragem não tão avançada dos textos.
Para remediar esse problema seria possível incorporar a livraria NLTK (https://www.geeksforgeeks.org/removing-stop-words-nltk-python/) no programa para agilizar a limpeza de stop-words, e juntamente a ela a livraria do scikit-learn (https://scikit-learn.org/dev/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html) para fazer uma aplicação confiavél do método do Tf-idf para criar um sistema de "penalização" nas palavras mais frequentenmente compartilhadas entre todas as categorias, assim removendo a necessidade de remover um número delas no processo de remoção de stop-words.

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**